In [12]:
import pandas as pd


- Após realizar o Sanity Checker, devemos realizar o EDA (Exploratory Data Analysis), o objetivo é evitar que o Scorecard seja construído em cima de dados quebrados

In [13]:
DATASET = "../data/cs-training.csv"
COLUMN_TARGET = "SeriousDlqin2yrs"
df = pd.read_csv(DATASET)
df.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [15]:
print("Shape:", df.shape)
print("Target rate:", df[COLUMN_TARGET].mean())


Shape: (150000, 12)
Target rate: 0.06684


In [ ]:
def main():
    df = pd.read_csv(DATASET)

# 1) Remover coluna lixo de índice

    df = df.drop(columns=["Unnamed: 0"])

    print("Shape:", df.shape)
    print("Target rate:", df[COLUMN_TARGET].mean())

# 2) Missing
    print("\nMissing por coluna:")
    print(df.isna().sum().sort_values(ascending=False).head(10))

# 3) estatísticas básicas (principalmente para ver extremos)
    cols = [c for c in df.columns if c != COLUMN_TARGET]
    print("\nDescribe para indetificação de outliers:")
    print(df[cols].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).T[["min","1%","5%","50%","95%","99%","max"]])
# Outlier identificado na idade

# 4) checagens valores suspeitos
    print("\nChecagens rápidas de valores suspeitos:")
    #Checarei idade pois identifiquei um outlier
    if "age" in df.columns:
        print("age <= 0:", (df["age"] <= 0).sum())
        print("age < 18:", (df["age"] < 18).sum())

# Foi identificado outliers no "RevolvingUtilizationOfUnsecuredLines" Essa coluna indica o quanto uma pessoa está fazendo
# utilização de crédito rotativo sem garantia. 

# Um exemplo é quando a fatura de um cartão de crédito não é paga inteiramente até o final do mês e é cobrado um juros em cima dela.

# O Valor é de 0 a 1, até pode ultrapassar 1, isso indica um uso de acima de 100% do limite, isso pode acontecer em alguns cenários 
# (juros, atrasos, limite reduzido, erro de cadastro) porém muito mais que isso pode indicar um outlier.

    if "RevolvingUtilizationOfUnsecuredLines" in df.columns:
        credito_rotativo = df["RevolvingUtilizationOfUnsecuredLines"]
        print("Utilization > 1:", (credito_rotativo > 1).sum())
        print("Utilization > 5:", (credito_rotativo > 5).sum())
        print("Utilization > 10:", (credito_rotativo > 10).sum())

# DebtRatio é as Dividas Mensais dividido pela renda mensal, segundo o dicionário do dataset está em porcentagem
# 30% da renda comprometida > Faz sentido
# 100% da renda comprometida (muito alto, mas possível)
# >= 200% gasta/paga 2x a renda por mês, bem improvável, mas pode acontecer se renda declarada está errada/baixa)
# >= 1000% quase sempre indica problema de dado
# >= 10000% impossível sem erro

    if "DebtRatio" in df.columns:
        debt_ratio = df["DebtRatio"]
        print("DebtRatio > 1:", (debt_ratio > 1).sum()) # >100% comprometido
        print("DebtRatio > 10:", (debt_ratio > 10).sum()) # >1000% comprometido
        print("DebtRatio > 100:", (debt_ratio > 100).sum()) # >10000% comprometido 

# Colunas de late_cols(atrasos): contam quantas vezes o cliente ficou atrasado em faixas de dias (30–59, 60–89 e 90+).
# Aqui a gente checa quantos registros têm valores muito altos (>=10 e >=90) para detectar outliers/códigos estranhos

    late_cols = [
        "NumberOfTime30-59DaysPastDueNotWorse",
        "NumberOfTime60-89DaysPastDueNotWorse",
        "NumberOfTimes90DaysLate",
    ]

    for c in late_cols:
        if c in df.columns:
            print(f"{c} >= 10:", (df[c] >= 10).sum())
            print(f"{c} >= 90:", (df[c] >= 90).sum())

if __name__ == "__main__":
    main()

Shape: (150000, 11)
Target rate: 0.06684

Missing por coluna:
MonthlyIncome                           29731
NumberOfDependents                       3924
SeriousDlqin2yrs                            0
age                                         0
RevolvingUtilizationOfUnsecuredLines        0
DebtRatio                                   0
NumberOfTime30-59DaysPastDueNotWorse        0
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
dtype: int64

Describe (pontos importantes p/ outliers):
                                      min    1%           5%          50%  \
RevolvingUtilizationOfUnsecuredLines  0.0   0.0     0.000000     0.154181   
age                                   0.0  24.0    29.000000    52.000000   
NumberOfTime30-59DaysPastDueNotWorse  0.0   0.0     0.000000     0.000000   
DebtRatio                             0.0   0.0     0.004329     0.366508   
MonthlyIncome                       

As colunas MonthlyIncome & NumberOfDependents não vão servir pois não estão confiáveis devido à quantidade de nulos.

Pessoas menores de 18 anos devem ser deletadas do modelo.